In [1]:
import sys
sys.path.append("C:/Users/user/meepc")
import numpy as np
import pandas as pd
from models import Hankel,Pipeline,Corrhankel
import warnings
warnings.simplefilter('ignore')
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
train_normal = pd.read_csv('~/data/te/normal_training.csv')
train_attack = pd.read_csv('~/data/te/attack_training.csv')

In [3]:
sensors = [col for col in train_normal.columns if col not in ['faultNumber', 'simulationRun', 'sample','LABEL']]

In [4]:
scaler = StandardScaler()
X_normal = pd.DataFrame(index=train_normal.index, columns=sensors, data=scaler.fit_transform(train_normal[sensors]))
X_attack = pd.DataFrame(data=scaler.fit_transform(train_attack.loc[:,sensors]),index=train_attack.index,columns=sensors).reset_index().drop(columns=['index'])

In [5]:
hankel = Hankel()
corrhankel = Corrhankel()
lag = 360
stride = 0.5

In [6]:
corr_normal,nolag_normal = corrhankel.fit(X_normal.to_numpy(),lag,stride)

In [7]:
corr_attack,nolag_attack = corrhankel.fit(X_attack.to_numpy(),lag,stride)

In [8]:
df_test1 = pd.read_csv('~/data/te/normal_testing.csv')
df_test2 = pd.read_csv('~/data/te/attack_testing.csv')
df_test=pd.concat((df_test1,df_test2),axis=0)

In [9]:
test_combined = pd.concat((df_test,train_attack),axis=0)
X_test = pd.DataFrame(index=test_combined.index, columns=sensors, data=scaler.fit_transform(test_combined[sensors]))
corr_test,nolag_test = corrhankel.fit(X_test.to_numpy(),lag,stride)
Y_test = test_combined.loc[:,'LABEL']

In [10]:
labels = hankel.fit(np.array(Y_test),lag,stride)
y_actual = np.any(labels>0,axis=0).astype(int)

# Epasad with 1 cluster and no threshold tuning (training attack included in test data)

In [11]:
sensor_models= []
sensor_predicted = []
accuracy = []
precision = []
recall = []
fscore = []
for i,sens in enumerate(sensors):
    train_normal = X_normal.loc[:,sens].values
    train_attack = X_attack.loc[:,sens].values
    model = Pipeline()
    model.fit(train_normal,train_attack,lag,stride,optimal_k=1,tune=False,kscore_init='inertia',corr_normal=corr_normal[:,i].reshape(nolag_normal,len(X_normal.columns)).T,only_corr=True)
    test = X_test.loc[:,sens].values
    y_predicted = model.predict(test,corr_test[:,i].reshape(nolag_test,len(X_test.columns)).T)
    sensor_predicted.append(y_predicted)
    accuracy.append(accuracy_score(y_actual,y_predicted))
    precision.append(precision_score(y_actual,y_predicted))
    recall.append(recall_score(y_actual,y_predicted)) 
    fscore.append(f1_score(y_actual,y_predicted))
    sensor_models.append(model)   

In [12]:
sensor_predicitions = np.asarray(sensor_predicted)
predicted_label=np.any(sensor_predicitions>0,axis=0).astype(int)
predicted_label

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [13]:
print("Accuracy ",accuracy_score(y_actual,predicted_label))
print("Precision ",precision_score(y_actual,predicted_label))
print("Recall ",recall_score(y_actual,predicted_label))
print("F1-score ",f1_score(y_actual,predicted_label))

Accuracy  0.9759036144578314
Precision  0.9759036144578314
Recall  1.0
F1-score  0.9878048780487805


In [14]:
print("Accuracy -  Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(accuracy).mean(), np.median(np.asarray(accuracy)),np.asarray(accuracy).min(), np.asarray(accuracy).max()))
print("precision - Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(precision).mean(), np.median(np.asarray(precision)),np.asarray(precision).min(), np.asarray(precision).max()))
print("recall -    Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(recall).mean(), np.median(np.asarray(recall)),np.asarray(recall).min(), np.asarray(recall).max()))
print("f1 -        Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(fscore).mean(), np.median(np.asarray(fscore)),np.asarray(fscore).min(), np.asarray(fscore).max()))

Accuracy -  Mean: 0.7957599629286378 Median : 0.9759036144578314 Min : 0.21686746987951808 Max : 0.9759036144578314
precision - Mean: 0.9817984837037633 Median : 0.9759036144578314 Min : 0.9724137931034482 Max : 1.0
recall -    Mean: 0.8087606837606838 Median : 1.0 Min : 0.19753086419753085 Max : 1.0
f1 -        Mean: 0.8477713063101251 Median : 0.9878048780487805 Min : 0.32989690721649484 Max : 0.9878048780487805


# Multiple clusters: no threshold tuning (training included in test data)

In [15]:
fscores_k_1 = []
for k in range(2,8):
    sensor_models= []
    sensor_predicted = []
    accuracy = []
    precision = []
    recall = []
    fscore = []
    for i,sens in enumerate(sensors):
        train_normal = X_normal.loc[:,sens].values
        train_attack = X_attack.loc[:,sens].values
        model = Pipeline()
        model.fit(train_normal,train_attack,lag,stride,optimal_k = k,tune=False,kscore_init='inertia',corr_normal=corr_normal[:,i].reshape(nolag_normal,len(X_normal.columns)).T,only_corr=True)
        test = X_test.loc[:,sens].values
        y_predicted = model.predict(test,corr_test=corr_test[:,i].reshape(nolag_test,len(X_test.columns)).T)
        sensor_predicted.append(y_predicted)
        accuracy.append(accuracy_score(y_actual,y_predicted))
        precision.append(precision_score(y_actual,y_predicted))
        recall.append(recall_score(y_actual,y_predicted)) 
        fscore.append(f1_score(y_actual,y_predicted))
        sensor_models.append(model)   
    sensor_predicted = np.asarray(sensor_predicted)
    y_predicted = np.any(sensor_predicted,axis=0).astype(int) 
    print("-------number of clusters------", k)
    print("Accuracy ",accuracy_score(y_actual,y_predicted))
    print("Precision ",precision_score(y_actual,y_predicted))
    print("Recall ",recall_score(y_actual,y_predicted))
    print("F1-score ",f1_score(y_actual,y_predicted))
    fscores_k_1.append(f1_score(y_actual,y_predicted))

-------number of clusters------ 2
Accuracy  0.9759036144578314
Precision  0.9759036144578314
Recall  1.0
F1-score  0.9878048780487805
-------number of clusters------ 3
Accuracy  0.9759036144578314
Precision  0.9759036144578314
Recall  1.0
F1-score  0.9878048780487805
-------number of clusters------ 4
Accuracy  0.9759036144578314
Precision  0.9759036144578314
Recall  1.0
F1-score  0.9878048780487805
-------number of clusters------ 5
Accuracy  0.9759036144578314
Precision  0.9759036144578314
Recall  1.0
F1-score  0.9878048780487805
-------number of clusters------ 6
Accuracy  0.9759036144578314
Precision  0.9759036144578314
Recall  1.0
F1-score  0.9878048780487805
-------number of clusters------ 7
Accuracy  0.9759036144578314
Precision  0.9759036144578314
Recall  1.0
F1-score  0.9878048780487805


In [16]:
fscores_k_1

[0.9878048780487805,
 0.9878048780487805,
 0.9878048780487805,
 0.9878048780487805,
 0.9878048780487805,
 0.9878048780487805]

# Epasad with 1 cluster : (training attack is not in test data)

In [17]:
X_test = pd.DataFrame(index=df_test.index, columns=sensors, data=scaler.fit_transform(df_test[sensors]))
Y_test = df_test.loc[:,'LABEL']
corr_test,nolag_test = corrhankel.fit(X_test.to_numpy(),lag,stride)

In [18]:
labels = hankel.fit(np.array(Y_test),lag,stride)
y_actual = np.any(labels>0,axis=0).astype(int)

In [19]:
sensor_models= []
sensor_predicted = []
accuracy = []
precision = []
recall = []
fscore = []
for i,sens in enumerate(sensors):
    train_normal = X_normal.loc[:,sens].values
    train_attack = X_attack.loc[:,sens].values
    model = Pipeline()
    model.fit(train_normal,train_attack,lag,stride,optimal_k=1,tune=False,kscore_init='inertia',corr_normal=corr_normal[:,i].reshape(nolag_normal,len(X_normal.columns)).T,corr_attack=corr_attack[:,i].reshape(nolag_attack,len(X_attack.columns)).T,only_corr=True)
    test = X_test.loc[:,sens].values
    y_predicted = model.predict(test,corr_test[:,i].reshape(nolag_test,len(X_test.columns)).T)
    sensor_predicted.append(y_predicted)
    accuracy.append(accuracy_score(y_actual,y_predicted))
    precision.append(precision_score(y_actual,y_predicted))
    recall.append(recall_score(y_actual,y_predicted)) 
    fscore.append(f1_score(y_actual,y_predicted))
    sensor_models.append(model)   

In [20]:
sensor_predicitions = np.asarray(sensor_predicted)
predicted_label=np.any(sensor_predicitions>0,axis=0).astype(int)
predicted_label

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [21]:
print("Accuracy ",accuracy_score(y_actual,predicted_label))
print("Precision ",precision_score(y_actual,predicted_label))
print("Recall ",recall_score(y_actual,predicted_label))
print("F1-score ",f1_score(y_actual,predicted_label))

Accuracy  0.9636363636363636
Precision  0.9636363636363636
Recall  1.0
F1-score  0.9814814814814815


In [22]:
print("Accuracy -  Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(accuracy).mean(), np.median(np.asarray(accuracy)),np.asarray(accuracy).min(), np.asarray(accuracy).max()))
print("precision - Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(precision).mean(), np.median(np.asarray(precision)),np.asarray(precision).min(), np.asarray(precision).max()))
print("recall -    Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(recall).mean(), np.median(np.asarray(recall)),np.asarray(recall).min(), np.asarray(recall).max()))
print("f1 -        Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(fscore).mean(), np.median(np.asarray(fscore)),np.asarray(fscore).min(), np.asarray(fscore).max()))

Accuracy -  Mean: 0.7825174825174823 Median : 0.9636363636363636 Min : 0.2 Max : 0.9636363636363636
precision - Mean: 0.9733124814142222 Median : 0.9636363636363636 Min : 0.9591836734693877 Max : 1.0
recall -    Mean: 0.8015239477503628 Median : 1.0 Min : 0.16981132075471697 Max : 1.0
f1 -        Mean: 0.8342871019079916 Median : 0.9814814814814815 Min : 0.2903225806451613 Max : 0.9814814814814815


# Multiple clusters : No threshold tuning (training attack is not in test data)

In [23]:
fscores_k_2=[]
for k in range(2,8):
    sensor_models = []
    sensor_predicted = []
    accuracy = []
    precision = []
    recall = []
    fscore = []
    for i,sens in enumerate(sensors):
        train_normal = X_normal.loc[:,sens].values
        train_attack = X_attack.loc[:,sens].values
        model = Pipeline()
        model.fit(train_normal,train_attack,lag,stride,optimal_k=k,tune=False,kscore_init='inertia',corr_normal=corr_normal[:,i].reshape(nolag_normal,len(X_normal.columns)).T,only_corr=True)
        test = X_test.loc[:,sens].values
        y_predicted = model.predict(test,corr_test=corr_test[:,i].reshape(nolag_test,len(X_normal.columns)).T)
        sensor_predicted.append(y_predicted)
        accuracy.append(accuracy_score(y_actual,y_predicted))
        precision.append(precision_score(y_actual,y_predicted))
        recall.append(recall_score(y_actual,y_predicted))
        fscore.append(f1_score(y_actual,y_predicted))
        sensor_models.append(model)    
    sensor_predicted = np.asarray(sensor_predicted)
    y_predicted = np.any(sensor_predicted,axis=0).astype(int)
    print("-------number of clusters------", k)
    print("Accuracy ",accuracy_score(y_actual,y_predicted))
    print("Precision ",precision_score(y_actual,y_predicted))
    print("Recall ",recall_score(y_actual,y_predicted))
    print("F1-score ",f1_score(y_actual,y_predicted))
    fscores_k_2.append(f1_score(y_actual,y_predicted))

-------number of clusters------ 2
Accuracy  0.9636363636363636
Precision  0.9636363636363636
Recall  1.0
F1-score  0.9814814814814815
-------number of clusters------ 3
Accuracy  0.9636363636363636
Precision  0.9636363636363636
Recall  1.0
F1-score  0.9814814814814815
-------number of clusters------ 4
Accuracy  0.9636363636363636
Precision  0.9636363636363636
Recall  1.0
F1-score  0.9814814814814815
-------number of clusters------ 5
Accuracy  0.9636363636363636
Precision  0.9636363636363636
Recall  1.0
F1-score  0.9814814814814815
-------number of clusters------ 6
Accuracy  0.9636363636363636
Precision  0.9636363636363636
Recall  1.0
F1-score  0.9814814814814815
-------number of clusters------ 7
Accuracy  0.9636363636363636
Precision  0.9636363636363636
Recall  1.0
F1-score  0.9814814814814815


In [24]:
fscores_k_2

[0.9814814814814815,
 0.9814814814814815,
 0.9814814814814815,
 0.9814814814814815,
 0.9814814814814815,
 0.9814814814814815]

# Multiple clusters : threshold tuning (training attack is not in test data)

In [25]:
fscores_k_3 = []
for k in range(2,8):
    sensor_models= []
    sensor_predicted = []
    accuracy = []
    precision = []
    recall = []
    fscore = []
    for i,sens in enumerate(sensors):
        train_normal = X_normal.loc[:,sens].values
        train_attack = X_attack.loc[:,sens].values
        model = Pipeline()
        model.fit(train_normal,train_attack,lag,stride,optimal_k = k,tune=True,kscore_init='inertia',corr_normal=corr_normal[:,i].reshape(nolag_normal,len(X_normal.columns)).T,corr_attack=corr_attack[:,i].reshape(nolag_attack,len(X_attack.columns)).T,only_corr=True)
        test = X_test.loc[:,sens].values
        y_predicted = model.predict(test,corr_test=corr_test[:,i].reshape(nolag_test,len(X_test.columns)).T)
        sensor_predicted.append(y_predicted)
        accuracy.append(accuracy_score(y_actual,y_predicted))
        precision.append(precision_score(y_actual,y_predicted))
        recall.append(recall_score(y_actual,y_predicted)) 
        fscore.append(f1_score(y_actual,y_predicted))
        sensor_models.append(model)   
    sensor_predicted = np.asarray(sensor_predicted)
    y_predicted = np.any(sensor_predicted,axis=0).astype(int)
    print("-------number of clusters------", k)
    print("Accuracy ",accuracy_score(y_actual,y_predicted))
    print("Precision ",precision_score(y_actual,y_predicted))
    print("Recall ",recall_score(y_actual,y_predicted))
    print("F1-score ",f1_score(y_actual,y_predicted))
    fscores_k_3.append(f1_score(y_actual,y_predicted))

-------number of clusters------ 2
Accuracy  0.9636363636363636
Precision  0.9636363636363636
Recall  1.0
F1-score  0.9814814814814815
-------number of clusters------ 3
Accuracy  0.9636363636363636
Precision  0.9636363636363636
Recall  1.0
F1-score  0.9814814814814815
-------number of clusters------ 4
Accuracy  0.9636363636363636
Precision  0.9636363636363636
Recall  1.0
F1-score  0.9814814814814815
-------number of clusters------ 5
Accuracy  0.9636363636363636
Precision  0.9636363636363636
Recall  1.0
F1-score  0.9814814814814815
-------number of clusters------ 6
Accuracy  0.9636363636363636
Precision  0.9636363636363636
Recall  1.0
F1-score  0.9814814814814815
-------number of clusters------ 7
Accuracy  0.9636363636363636
Precision  0.9636363636363636
Recall  1.0
F1-score  0.9814814814814815


In [26]:
fscores_k_3

[0.9814814814814815,
 0.9814814814814815,
 0.9814814814814815,
 0.9814814814814815,
 0.9814814814814815,
 0.9814814814814815]